# 메타클래스로 서브클래스를 검증하자

메타클ㄹ래스는 서브클래스가 정의될 때마다 검증 코드를 실행하는 신뢰할 만한 방법을 제공한다.  
보통 클래스 검증 코드는 클래스의 객체가 생성될 때 `__init__`메서드에서 실행된다. 메타클래스를 검증용으로 사용하면 오류를 더 빨리 일으킬 수 있다.

서브클래스 검증용으로 메타클래스를 정의하는 방법을 알아보기에 앞서 메타클래스가 표준 객체에는 어떻게 동작하는지 이해해야 한다.

메타 클래스는 type을 상속하여 정의한다. 메타클래스는 기본으로 자체의 `__new__` 메서드에서 연관된 class 문의 콘텐츠를 받는다. 여기서 타입이 실제로 생성되기 전에 클래스 정보를 수정할 수 있다.

In [2]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(object, metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass

(<class '__main__.Meta'>, 'MyClass', (<class 'object'>,), {'__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123, 'foo': <function MyClass.foo at 0x7fb810094160>})


클래스가 정의되기 전에 클래스의 모든 파라미터를 검증하려면 `Meta.__new__` 메서드에 기능을 추가하면 된다.

In [5]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # 추상 Polygon 클래스는 검증하지 않음
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
        
class Polygon(object, metaclass=ValidatePolygon):
    sides = None # 서브클래스에서 설정
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Polygon):
    sides = 3

In [6]:
print('Before class')
class Line(Polygon):
    print('Before sides')
    sides = 1
    print('After sides')
print('After class')

Before class
Before sides
After sides


ValueError: Polygons need 3+ sides